# Chapter 6: Agents 실습

이 노트북은 LangGraph를 사용한 에이전트 구현을 실습합니다.

## 환경 설정

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = input("OpenAI API Key를 입력하세요: ")

## 1. 기본 에이전트 구현

In [ ]:
from typing import TypedDict, Annotated, Sequence, Literal
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END, MessagesState
from langgraph.prebuilt import ToolNode
import operator
import json

# 도구 정의
@tool
def calculate(expression: str) -> str:
    """수식을 계산합니다. 예: '2 + 2', '10 * 5', '100 / 4'"""
    try:
        result = eval(expression)
        return f"계산 결과: {result}"
    except Exception as e:
        return f"계산 오류: {str(e)}"

@tool
def get_weather(city: str) -> str:
    """특정 도시의 날씨 정보를 가져옵니다."""
    # 실제로는 API를 호출하겠지만, 여기서는 시뮬레이션
    weather_data = {
        "서울": "맑음, 온도: 25°C, 습도: 60%",
        "부산": "흐림, 온도: 22°C, 습도: 70%",
        "제주": "비, 온도: 20°C, 습도: 85%",
        "뉴욕": "맑음, 온도: 18°C, 습도: 55%"
    }
    return weather_data.get(city, f"{city}의 날씨 정보를 찾을 수 없습니다.")

@tool
def search_web(query: str) -> str:
    """웹에서 정보를 검색합니다."""
    # 시뮬레이션된 검색 결과
    results = {
        "python": "Python은 1991년에 개발된 고급 프로그래밍 언어입니다.",
        "langchain": "LangChain은 LLM 애플리케이션 개발 프레임워크입니다.",
        "ai": "인공지능은 인간의 지능을 모방한 컴퓨터 시스템입니다."
    }
    
    for key, value in results.items():
        if key.lower() in query.lower():
            return value
    return f"'{query}'에 대한 검색 결과를 찾을 수 없습니다."

# 에이전트 State
class AgentState(MessagesState):
    pass

# 도구 호출 결정 함수
def should_continue(state: AgentState) -> Literal["tools", "end"]:
    messages = state["messages"]
    last_message = messages[-1]
    
    # AI 메시지에 tool_calls가 있으면 도구 실행
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return "end"

# 에이전트 노드
def call_model(state: AgentState):
    messages = state["messages"]
    
    # LLM 설정 (도구 바인딩)
    tools = [calculate, get_weather, search_web]
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    llm_with_tools = llm.bind_tools(tools)
    
    # 시스템 메시지 추가
    system_message = {
        "role": "system",
        "content": "당신은 도움이 되는 AI 어시스턴트입니다. 필요한 경우 도구를 사용하여 정확한 정보를 제공하세요."
    }
    
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

# 에이전트 그래프 생성
def create_basic_agent():
    workflow = StateGraph(AgentState)
    
    # 도구 노드 생성
    tools = [calculate, get_weather, search_web]
    tool_node = ToolNode(tools)
    
    # 노드 추가
    workflow.add_node("agent", call_model)
    workflow.add_node("tools", tool_node)
    
    # 엣지 추가
    workflow.set_entry_point("agent")
    
    # 조건부 엣지
    workflow.add_conditional_edges(
        "agent",
        should_continue,
        {
            "tools": "tools",
            "end": END
        }
    )
    
    # 도구 실행 후 다시 에이전트로
    workflow.add_edge("tools", "agent")
    
    return workflow.compile()

# 에이전트 실행
agent = create_basic_agent()

# 테스트
test_queries = [
    "2024년은 윤년인가요? 계산해서 알려주세요.",
    "서울의 날씨는 어때?",
    "25 * 4 + 10은 얼마인가요?",
    "LangChain에 대해 검색해주세요."
]

for query in test_queries:
    print(f"\n질문: {query}")
    result = agent.invoke({"messages": [HumanMessage(content=query)]})
    print(f"답변: {result['messages'][-1].content}")

## 2. 첫 도구 강제 실행 에이전트

In [ ]:
from typing import Dict, List

# 검색을 먼저 수행하는 에이전트 State
class ForceFirstToolState(MessagesState):
    first_tool_executed: bool
    search_results: str

@tool
def research_topic(topic: str) -> str:
    """주제에 대한 상세 연구를 수행합니다."""
    research_data = {
        "quantum": "양자 컴퓨팅은 양자역학 원리를 활용한 혁신적인 컴퓨팅 기술입니다.",
        "blockchain": "블록체인은 분산 원장 기술로 거래 기록을 안전하게 저장합니다.",
        "metaverse": "메타버스는 가상과 현실이 융합된 3차원 디지털 공간입니다."
    }
    
    for key, value in research_data.items():
        if key in topic.lower():
            return f"연구 결과: {value}"
    return f"'{topic}'에 대한 연구 자료를 준비 중입니다."

@tool
def analyze_data(data: str) -> str:
    """데이터를 분석하고 인사이트를 도출합니다."""
    word_count = len(data.split())
    char_count = len(data)
    return f"분석 결과: 단어 수 {word_count}개, 문자 수 {char_count}개"

# 첫 도구 실행 노드
def force_first_tool(state: ForceFirstToolState):
    """첫 번째로 research_topic 도구를 강제 실행"""
    if not state.get("first_tool_executed", False):
        messages = state["messages"]
        query = messages[-1].content if messages else ""
        
        # 쿼리에서 주제 추출
        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
        extract_prompt = f"다음 질문에서 핵심 주제를 한 단어로 추출하세요: {query}"
        topic = llm.invoke(extract_prompt).content
        
        # research_topic 도구 실행
        research_result = research_topic.invoke({"topic": topic})
        
        state["first_tool_executed"] = True
        state["search_results"] = research_result
        
        # 결과를 메시지에 추가
        ai_message = AIMessage(content=f"먼저 '{topic}'에 대해 조사했습니다: {research_result}")
        return {"messages": [ai_message], "first_tool_executed": True, "search_results": research_result}
    
    return state

# 메인 에이전트 노드
def process_with_context(state: ForceFirstToolState):
    """연구 결과를 바탕으로 응답 생성"""
    messages = state["messages"]
    search_results = state.get("search_results", "")
    
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
    tools = [analyze_data]
    llm_with_tools = llm.bind_tools(tools)
    
    # 컨텍스트를 포함한 프롬프트
    context_message = f"""
    사용자 질문에 답하세요. 다음 연구 결과를 참고하세요:
    {search_results}
    
    필요하면 analyze_data 도구를 사용할 수 있습니다.
    """
    
    enhanced_messages = [
        {"role": "system", "content": context_message}
    ] + messages
    
    response = llm_with_tools.invoke(enhanced_messages)
    return {"messages": [response]}

# 첫 도구 강제 실행 에이전트 생성
def create_force_first_tool_agent():
    workflow = StateGraph(ForceFirstToolState)
    
    # 노드 추가
    workflow.add_node("force_research", force_first_tool)
    workflow.add_node("process", process_with_context)
    workflow.add_node("tools", ToolNode([analyze_data]))
    
    # 플로우 정의
    workflow.set_entry_point("force_research")
    workflow.add_edge("force_research", "process")
    
    # 조건부 엣지
    def check_tool_calls(state):
        messages = state["messages"]
        last_message = messages[-1]
        if hasattr(last_message, "tool_calls") and last_message.tool_calls:
            return "tools"
        return "end"
    
    workflow.add_conditional_edges(
        "process",
        check_tool_calls,
        {
            "tools": "tools",
            "end": END
        }
    )
    
    workflow.add_edge("tools", "process")
    
    return workflow.compile()

# 실행
force_first_agent = create_force_first_tool_agent()

queries = [
    "양자 컴퓨팅에 대해 설명해주세요.",
    "블록체인 기술의 미래는?",
    "메타버스란 무엇인가요?"
]

for query in queries:
    print(f"\n질문: {query}")
    print("="*60)
    result = force_first_agent.invoke({
        "messages": [HumanMessage(content=query)],
        "first_tool_executed": False
    })
    
    for msg in result["messages"][-2:]:
        if isinstance(msg, AIMessage):
            print(f"AI: {msg.content}")

## 3. 다중 도구 조합 에이전트

In [ ]:
from datetime import datetime
import random

# 다양한 도구들 정의
@tool
def get_current_time() -> str:
    """현재 시간을 반환합니다."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

@tool
def generate_random_number(min_val: int, max_val: int) -> int:
    """지정된 범위 내의 랜덤 숫자를 생성합니다."""
    return random.randint(min_val, max_val)

@tool
def translate_text(text: str, target_language: str) -> str:
    """텍스트를 다른 언어로 번역합니다."""
    translations = {
        "english": {
            "안녕하세요": "Hello",
            "감사합니다": "Thank you",
            "좋은 하루 되세요": "Have a nice day"
        },
        "japanese": {
            "안녕하세요": "こんにちは",
            "감사합니다": "ありがとうございます",
            "좋은 하루 되세요": "良い一日を"
        }
    }
    
    lang_dict = translations.get(target_language.lower(), {})
    return lang_dict.get(text, f"번역할 수 없습니다: {text}")

@tool
def save_note(title: str, content: str) -> str:
    """노트를 저장합니다."""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    return f"노트 '{title}'가 저장되었습니다. ID: note_{timestamp}"

@tool
def get_stock_price(symbol: str) -> dict:
    """주식 가격 정보를 가져옵니다."""
    # 시뮬레이션된 주식 데이터
    stocks = {
        "AAPL": {"price": 175.50, "change": "+2.3%"},
        "GOOGL": {"price": 142.80, "change": "-0.5%"},
        "MSFT": {"price": 380.20, "change": "+1.2%"},
        "TSLA": {"price": 242.60, "change": "+3.8%"}
    }
    return stocks.get(symbol, {"error": f"주식 심볼 {symbol}을 찾을 수 없습니다."})

@tool
def create_todo(task: str, priority: str = "medium") -> str:
    """할 일을 생성합니다."""
    priorities = ["low", "medium", "high"]
    if priority not in priorities:
        priority = "medium"
    
    todo_id = random.randint(1000, 9999)
    return f"할 일 생성됨 - ID: {todo_id}, 작업: '{task}', 우선순위: {priority}"

# 복잡한 작업 처리 State
class ComplexAgentState(MessagesState):
    tool_history: List[str]
    task_complexity: str
    intermediate_results: Dict

# 작업 분석 노드
def analyze_task(state: ComplexAgentState):
    """작업의 복잡도를 분석하고 필요한 도구 결정"""
    messages = state["messages"]
    if not messages:
        return state
    
    query = messages[-1].content.lower()
    
    # 복잡도 판단
    tool_keywords = {
        "시간": ["get_current_time"],
        "랜덤": ["generate_random_number"],
        "번역": ["translate_text"],
        "주식": ["get_stock_price"],
        "할일": ["create_todo"],
        "노트": ["save_note"]
    }
    
    required_tools = []
    for keyword, tools in tool_keywords.items():
        if keyword in query:
            required_tools.extend(tools)
    
    complexity = "simple" if len(required_tools) <= 1 else "complex"
    
    state["task_complexity"] = complexity
    state["tool_history"] = []
    state["intermediate_results"] = {}
    
    print(f"작업 복잡도: {complexity}")
    print(f"예상 필요 도구: {required_tools}")
    
    return state

# 복합 도구 실행 노드
def execute_complex_task(state: ComplexAgentState):
    """복잡한 작업을 위해 여러 도구 조합"""
    messages = state["messages"]
    
    # 모든 도구
    all_tools = [
        get_current_time, generate_random_number, translate_text,
        save_note, get_stock_price, create_todo, calculate, get_weather
    ]
    
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    llm_with_tools = llm.bind_tools(all_tools)
    
    # 시스템 프롬프트
    system_prompt = f"""
    당신은 다양한 도구를 활용하는 고급 AI 어시스턴트입니다.
    작업 복잡도: {state.get('task_complexity', 'unknown')}
    이미 사용한 도구: {state.get('tool_history', [])}
    
    필요한 모든 도구를 적절히 조합하여 사용자 요청을 완벽하게 처리하세요.
    """
    
    enhanced_messages = [
        {"role": "system", "content": system_prompt}
    ] + messages
    
    response = llm_with_tools.invoke(enhanced_messages)
    
    # 도구 사용 기록
    if hasattr(response, "tool_calls") and response.tool_calls:
        for tool_call in response.tool_calls:
            if "tool_history" not in state:
                state["tool_history"] = []
            state["tool_history"].append(tool_call["name"])
    
    return {"messages": [response]}

# 복합 에이전트 생성
def create_complex_agent():
    workflow = StateGraph(ComplexAgentState)
    
    # 모든 도구
    all_tools = [
        get_current_time, generate_random_number, translate_text,
        save_note, get_stock_price, create_todo, calculate, get_weather
    ]
    
    # 노드 추가
    workflow.add_node("analyze", analyze_task)
    workflow.add_node("execute", execute_complex_task)
    workflow.add_node("tools", ToolNode(all_tools))
    
    # 플로우 정의
    workflow.set_entry_point("analyze")
    workflow.add_edge("analyze", "execute")
    
    # 조건부 엣지
    def router(state):
        messages = state["messages"]
        last_message = messages[-1]
        
        if hasattr(last_message, "tool_calls") and last_message.tool_calls:
            return "tools"
        
        # 최대 5번의 도구 호출 후 종료
        if len(state.get("tool_history", [])) > 5:
            return "end"
        
        return "end"
    
    workflow.add_conditional_edges(
        "execute",
        router,
        {
            "tools": "tools",
            "end": END
        }
    )
    
    workflow.add_edge("tools", "execute")
    
    return workflow.compile()

# 실행
complex_agent = create_complex_agent()

# 복잡한 작업 테스트
complex_queries = [
    "현재 시간을 알려주고, 1부터 100 사이의 랜덤 숫자를 생성해주세요.",
    "AAPL과 MSFT의 주식 가격을 확인하고, 더 높은 가격의 주식을 메모로 저장해주세요.",
    "'안녕하세요'를 영어와 일본어로 번역하고, 번역 결과를 할 일로 만들어주세요.",
    "서울 날씨를 확인하고, 날씨가 좋으면 '산책하기' 할 일을 높은 우선순위로 만들어주세요."
]

for query in complex_queries:
    print(f"\n{'='*80}")
    print(f"질문: {query}\n")
    
    result = complex_agent.invoke({
        "messages": [HumanMessage(content=query)]
    })
    
    # 최종 응답 출력
    final_message = result["messages"][-1]
    if isinstance(final_message, AIMessage):
        print(f"\n최종 답변:\n{final_message.content}")
    
    # 사용된 도구 목록
    if result.get("tool_history"):
        print(f"\n사용된 도구: {', '.join(result['tool_history'])}")

## 4. 자율 계획 에이전트

In [ ]:
from pydantic import BaseModel, Field
from typing import List

# 계획 스키마
class TaskPlan(BaseModel):
    steps: List[str] = Field(description="실행할 단계들")
    tools_needed: List[str] = Field(description="필요한 도구들")
    expected_output: str = Field(description="예상 결과")

# 자율 에이전트 State
class AutonomousAgentState(MessagesState):
    plan: TaskPlan
    current_step: int
    step_results: List[str]
    final_summary: str

# 계획 수립 노드
def create_plan(state: AutonomousAgentState):
    """작업 계획 수립"""
    messages = state["messages"]
    query = messages[-1].content if messages else ""
    
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    structured_llm = llm.with_structured_output(TaskPlan)
    
    planning_prompt = f"""
    다음 작업을 수행하기 위한 상세한 계획을 수립하세요:
    
    작업: {query}
    
    사용 가능한 도구:
    - calculate: 수식 계산
    - get_weather: 날씨 정보
    - search_web: 웹 검색
    - get_current_time: 현재 시간
    - save_note: 노트 저장
    
    단계별로 명확하게 계획을 세우세요.
    """
    
    plan = structured_llm.invoke(planning_prompt)
    
    state["plan"] = plan
    state["current_step"] = 0
    state["step_results"] = []
    
    print("수립된 계획:")
    for i, step in enumerate(plan.steps, 1):
        print(f"{i}. {step}")
    print(f"\n필요 도구: {', '.join(plan.tools_needed)}")
    print(f"예상 결과: {plan.expected_output}\n")
    
    return state

# 단계 실행 노드
def execute_step(state: AutonomousAgentState):
    """계획의 각 단계 실행"""
    plan = state.get("plan")
    if not plan:
        return state
    
    current_step = state.get("current_step", 0)
    if current_step >= len(plan.steps):
        return state
    
    step_description = plan.steps[current_step]
    print(f"실행 중: 단계 {current_step + 1} - {step_description}")
    
    # 도구 바인딩
    tools = [calculate, get_weather, search_web, get_current_time, save_note]
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    llm_with_tools = llm.bind_tools(tools)
    
    # 단계 실행 프롬프트
    step_prompt = f"""
    현재 단계: {step_description}
    이전 결과들: {state.get('step_results', [])}
    
    이 단계를 수행하기 위해 적절한 도구를 사용하세요.
    """
    
    response = llm_with_tools.invoke(step_prompt)
    
    # 결과 저장
    if "step_results" not in state:
        state["step_results"] = []
    state["step_results"].append(f"단계 {current_step + 1} 완료: {response.content[:100]}")
    state["current_step"] = current_step + 1
    
    return {"messages": [response], "current_step": current_step + 1}

# 결과 종합 노드
def summarize_results(state: AutonomousAgentState):
    """모든 단계 결과 종합"""
    plan = state.get("plan")
    step_results = state.get("step_results", [])
    
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
    
    summary_prompt = f"""
    작업 계획:
    {plan.steps if plan else []}
    
    실행 결과:
    {step_results}
    
    예상했던 결과:
    {plan.expected_output if plan else ''}
    
    위 정보를 바탕으로 전체 작업의 최종 요약을 작성하세요.
    """
    
    summary = llm.invoke(summary_prompt)
    state["final_summary"] = summary.content
    
    return {"messages": [AIMessage(content=summary.content)]}

# 자율 에이전트 생성
def create_autonomous_agent():
    workflow = StateGraph(AutonomousAgentState)
    
    # 도구 노드
    tools = [calculate, get_weather, search_web, get_current_time, save_note]
    tool_node = ToolNode(tools)
    
    # 노드 추가
    workflow.add_node("plan", create_plan)
    workflow.add_node("execute", execute_step)
    workflow.add_node("tools", tool_node)
    workflow.add_node("summarize", summarize_results)
    
    # 플로우 정의
    workflow.set_entry_point("plan")
    workflow.add_edge("plan", "execute")
    
    # 단계 실행 라우터
    def step_router(state):
        messages = state["messages"]
        last_message = messages[-1] if messages else None
        
        # 도구 호출이 있으면 도구 실행
        if last_message and hasattr(last_message, "tool_calls") and last_message.tool_calls:
            return "tools"
        
        # 모든 단계 완료 확인
        plan = state.get("plan")
        current_step = state.get("current_step", 0)
        
        if plan and current_step >= len(plan.steps):
            return "summarize"
        
        return "execute"
    
    workflow.add_conditional_edges(
        "execute",
        step_router,
        {
            "tools": "tools",
            "execute": "execute",
            "summarize": "summarize"
        }
    )
    
    workflow.add_edge("tools", "execute")
    workflow.add_edge("summarize", END)
    
    return workflow.compile()

# 실행
autonomous_agent = create_autonomous_agent()

# 복잡한 작업 테스트
complex_tasks = [
    "서울의 날씨를 확인하고, 날씨에 따른 오늘의 활동 계획을 세워서 노트에 저장해주세요.",
    "현재 시간을 확인하고, 오후 3시까지 남은 시간을 계산한 다음, 그 시간 동안 할 수 있는 작업 목록을 만들어주세요."
]

for task in complex_tasks:
    print(f"\n{'='*80}")
    print(f"작업: {task}\n")
    
    result = autonomous_agent.invoke({
        "messages": [HumanMessage(content=task)]
    })
    
    print(f"\n최종 요약:")
    print(result.get("final_summary", "요약 없음"))

## 실습 과제

1. 외부 API와 연동하는 실제 에이전트 구현
2. 에러 처리와 재시도 로직이 있는 안정적인 에이전트
3. 학습 기능이 있는 적응형 에이전트 구현

In [ ]:
# 여기에 실습 코드를 작성하세요
